In [ ]:
using Optics_in_the_length_gauge
using CairoMakie # backend for plots

In [ ]:
cp = Computation_presets(        # comp presets
    xbounds = [0, 2π/√3],        # adimensional, 
    ybounds = [-2π, 2π],         # adimensional, 
    ωlist = collect(-10:0.1:10), # in eV, 
    broadening = 0.025,          # in eV
    evals = 1e6                  # number of iterations
    )

Quick note: 

Structs in Optics_in_the_lenght_gauge uses Parameters,
so they can be dynamically created using the following syntax.
cp2 = Computation_presets(cp, evals = 1e5), which keeps the rest
of fields as in cp. 

### Bands MLG

<img src="MLG_bands.jpg" alt="Alt text" width="400"/>

### DOS

In [ ]:

μ = 0
h(q) = Presets.MLG_hamiltonian(μ, q) # system specific

# DOS_presets
dos_presets = DOS_presets(
    h = h, 
    computation = cp            #
)

In [ ]:
# Calculation
ω, j = dos(dos_presets)

In [ ]:
fig = Figure(); ax = Axis(fig[1,1], xlabel = "ω (eV)", ylabel = "dos (a.u.)")
lines!(ax, ω, j)

<img src="MLG_dos.jpg" alt="Alt text" width="400"/>

Note that the broadening, $\eta\neq0$ introduces a finite spectral weight at $\omega = 0$ and tails at the band edges

### JDOS

In [ ]:
# Calculation, at two Fermi levels
h0(q) = Presets.MLG_hamiltonian(μ, q)
ω, j0 = jdos(JDOS_presets(h0, cp))

h1(q) = Presets.MLG_hamiltonian(μ + 1, q)
ω, j0 = jdos(JDOS_presets(h1, cp))

In [ ]:
fig = Figure(); ax = Axis(fig[1,1], xlabel = "ω (eV)", ylabel = "jdos (a.u.)")
lines!(ax, ω, j0, label = "μ = 0")
lines!(ax, ω, j1, label = "μ = 1 eV")
axislegend()
fig

<img src="MLG_jdos.jpg" alt="Alt text" width="400"/>

## Linear optical conductivity

### Example 1: $\sigma_{xx}$ of spinless MLG

In [ ]:
# Presets
n = 1
ϵ2 = 2π/(3√3)/n
ncp = Computation_presets(cp, 
    xbounds = Presets.K1[1] .+ [-ϵ2, ϵ2],
    ybounds = 2*abs(Presets.K1[1]) .* [-1, 1],
    ωlist = collect(0:.1:10))

μ = 0
h(q) = Presets.MLG_hamiltonian(μ, q)
nabla_h(q) = Presets.MLG_nabla(q) 
# vector containing the partial derivatives of the Hamiltonian along the periodic directions
sigma_ij_presets = σij_presets(dirJ = :x, dirE = :x, h, nabla_h, ncp)

In [ ]:
ω, j = linear_optical_conductivity(sigma_ij_presets)

In [ ]:
fig = Figure(); ax = Axis(fig[1,1], xlabel = "ω (eV)", ylabel = "σ_xx [e^2/16ħ]")
lines!(ax, ω, 16 .* j) 
fig


<img src="Optical_conductivity_large_frequency_range.jpg" alt="Alt text" width="400"/>

### Example 2: $\sigma_{xx}$ of a single Dirac cone

Here we can use the flexibility of `Computation_presets` to pass different k-integration
boundaries. Our MLG model covers the two valleys, but we can select `xbounds` and `ybounds` 
in the following manner to describe the low-energy (<1 eV) response of the K cone alone.

In [ ]:
# Calculation of the quantized conductivity of a single Dirac cone
n = 5
ϵ2 = 2π/(3√3)/n

cp_K_valley = Computation_presets(cp,         # updating computation presets
                xbounds = Presets.K1[1] .+ [-ϵ2, ϵ2], 
                ybounds = Presets.K1[2] .+ [-ϵ2, ϵ2],
                ωlist = collect(0:.025:0.5),
                broadening = 0.01)

# update σij_presets with cp_K_valley:
new_sigma_ij_presets = σij_presets(dirJ, dirE, h, nabla_h, cp_K_valley)

In [ ]:
# Calculation
ω, j_single_DC = linear_optical_conductivity(new_sigma_ij_presets)
# same calculation with smaller broadening:
ω, j_single_DC_2 = linear_optical_conductivity(σij_presets(new_sigma_ij_presets, broadening = 0.005))

In [ ]:
# Plot
fig = Figure(); ax = Axis(fig[1,1], xlabel = "ω (eV)", ylabel = "σ_xx [e^2/16ħ]")
lines!(ax, ω, 16 .* j_single_DC, label = "η = 5 meV") 
lines!(ax, ω, 16 .* j_single_DC_2, label = "η = 10 meV")
axislegend(position = :rb)
# the 16 prefactor is to express the conductivity
# in as e^2/(16ħ): σxx value of a single Dirac cone

<img src="quantized_conductivity_singleCone.jpg" alt="Alt text" width="400"/>

$\sigma_{xx} = \frac{e^2}{16\hbar}$ as expected for a interband processes of a single Dirac cone